In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install tensorflow==1.15
!pip install keras==2.2.5
!pip install git+https://www.github.com/keras-team/keras-contrib.git

  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-4xrm2phh
  Running command git clone -q https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-4xrm2phh
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-cp37-none-any.whl size=101065 sha256=c9288589bcecf46ac8cff2b9976b1a75596d15663138239e8e023ce496251e05
  Stored in directory: /tmp/pip-ephem-wheel-cache-y2to_4md/wheels/11/27/c8/4ed56de7b55f4f61244e2dc6ef3cdbaff2692527a2ce6502ba
Successfully built keras-contrib


In [ ]:
import numpy as np

In [ ]:
char_encode = np.loadtxt('/content/drive/MyDrive/fast_text/train/char_encode.txt')
print(char_encode.shape)
char_encode = char_encode.reshape(int(char_encode.shape[0]/42), 42, char_encode.shape[1])
print(char_encode.shape)


(132174, 32)
(3147, 42, 32)


In [ ]:
tag = np.loadtxt('/content/drive/MyDrive/fast_text/train/tag_embedd.txt')
tag = tag.reshape(int(tag.shape[0]/42), 42, tag.shape[1])
print(tag.shape)

(3147, 42, 13)


In [ ]:
word = np.loadtxt('/content/drive/MyDrive/fast_text/train/word_embedd.txt')
word = word.reshape(int(word.shape[0]/42), 42, word.shape[1])
print(word.shape)

(3147, 42, 300)


In [ ]:
# begin = 0
# end = 5957
# begin = 5958
# end = len(word)
# char_encode = char_encode[begin:end+1]
# print(char_encode.shape)
# tag = tag[begin:end+1]
# print(tag.shape)
# word = word[begin:end+1]
# print(word.shape)


In [ ]:
val_char_encode = np.loadtxt('/content/drive/MyDrive/fast_text/val/char_encode.txt')
print(val_char_encode.shape)
val_char_encode = val_char_encode.reshape(int(val_char_encode.shape[0]/42), 42, val_char_encode.shape[1])
print(val_char_encode.shape)
val_tag = np.loadtxt('/content/drive/MyDrive/fast_text/val/tag_embedd.txt')
val_tag = val_tag.reshape(int(val_tag.shape[0]/42), 42, val_tag.shape[1])
print(tag.shape)
val_word = np.loadtxt('/content/drive/MyDrive/fast_text/val/word_embedd.txt')
val_word = val_word.reshape(int(val_word.shape[0]/42), 42, val_word.shape[1])
print(val_word.shape)

(18900, 32)
(450, 42, 32)
(3147, 42, 13)
(450, 42, 300)


In [ ]:
# chuyển từ char encode sang char embedding
LEN_OF_VOCAB = 137
shape = char_encode.shape
char_embedd = np.zeros([shape[0],shape[1],shape[2],LEN_OF_VOCAB])
for i in range(shape[0]):
  for j in range(shape[1]):
    for k in range(shape[2]):
      char_int = char_encode[i,j,k]
      char_int = char_int.astype(np.int64)
      onehot = np.zeros(LEN_OF_VOCAB)
      onehot[char_int] = 1
      char_embedd[i,j,k,:] = onehot

print(char_embedd.shape)

shape = val_char_encode.shape
val_char_embedd = np.zeros([shape[0],shape[1],shape[2],LEN_OF_VOCAB])
for i in range(shape[0]):
  for j in range(shape[1]):
    for k in range(shape[2]):
      char_int = val_char_encode[i,j,k]
      char_int = char_int.astype(np.int64)
      onehot = np.zeros(LEN_OF_VOCAB)
      onehot[char_int] = 1
      val_char_embedd[i,j,k,:] = onehot

print(val_char_embedd.shape)

(3147, 42, 32, 137)
(450, 42, 32, 137)


In [ ]:
from keras.models import Sequential, Model, Input
from keras.layers import LSTM, Dense, TimeDistributed, Activation, Bidirectional, Masking, Embedding, Dropout, Flatten, concatenate, Conv1D, MaxPool1D, Lambda, Softmax
from keras import backend as K
from keras_contrib.layers import CRF
from keras.utils import plot_model
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras_contrib.losses import  crf_loss
from keras_contrib.metrics import crf_accuracy

Using TensorFlow backend.


In [ ]:
# hàm này trả về mảng các số k liên tiếp trong mảng X
def arr_tag_in_a_tag_sen(sen_pred, sen_true, tag):
  j=0
  bat_dau=-1
  ket_thuc=-1
  while j < len(sen_pred):
    if sen_true[j]==tag:
      bat_dau=j
      ket_thuc=j
      while sen_true[ket_thuc+1]==1:
        ket_thuc=ket_thuc+1
      ket_thuc=ket_thuc+1
      break
    else:
        j=j+1

  if bat_dau==-1 and ket_thuc==-1:
    return 0
  
  a = sen_pred[bat_dau:ket_thuc]
  b = sen_true[bat_dau:ket_thuc]
  a = np.array(a)
  b = np.array(b)
  if (a==b).all():
    return 1
  else:
    return 0

In [ ]:
# def acc1(y_true, y_pred):
#   y_true = tf. y_true.numpy()
#   y_pred = y_pred.numpy()
#   y_pred = y_pred.reshape(y_pred.shape[0]*y_pred.shape[1], y_pred.shape[2])
#   y_pred = np.argmax(y_pred,axis=1)
#   y_true = y_true.reshape(y_true.shape[0]*y_true.shape[1], y_true.shape[2])
#   y_true = np.argmax(y_true,axis=1)
#   acc=0
#   false=0

#   homenumber_true = 0
#   street_true = 0 
#   ward_true = 0
#   district_true = 0
#   province_true = 0
#   country_true = 0
#   postcode_true = 0
#   ner_true = 0
#   obj_true = 0
#   obj_feature_true = 0
#   pre_true = 0

#   homenumber_false = 0
#   street_false = 0
#   ward_false = 0
#   district_false = 0
#   province_false = 0
#   country_false = 0
#   postcode_false = 0
#   ner_false = 0
#   obj_false = 0
#   obj_feature_false = 0
#   pre_false = 0
 

#   if arr_tag_in_a_tag_sen(sen_pred, sen_true, 1)==1:
#     homenumber_true+=1
#   else:
#     homenumber_false+=1
  
#   if arr_tag_in_a_tag_sen(sen_pred, sen_true, 1)==1:
#     homenumber_true+=1
#   else:
#     homenumber_false+=1

#   if arr_tag_in_a_tag_sen(sen_pred, sen_true, 2)==1:
#     street_true+=1
#   else:
#     street_false+=1
  
#   if arr_tag_in_a_tag_sen(sen_pred, sen_true, 3)==1:
#     ward_true+=1
#   else:
#     ward_false+=1
  
#   if arr_tag_in_a_tag_sen(sen_pred, sen_true, 4)==1:
#     district_true+=1
#   else:
#     district_false+=1
  
#   if arr_tag_in_a_tag_sen(sen_pred, sen_true, 5)==1:
#     province_true+=1
#   else:
#     province_false+=1
  
#   if arr_tag_in_a_tag_sen(sen_pred, sen_true, 6)==1:
#     country_true+=1
#   else:
#     country_false+=1
  
#   if arr_tag_in_a_tag_sen(sen_pred, sen_true, 7)==1:
#     postcode_true+=1
#   else:
#     postcode_false+=1
  
#   if arr_tag_in_a_tag_sen(sen_pred, sen_true, 8)==1:
#     ner_true+=1
#   else:
#     ner_false+=1
  
#   if arr_tag_in_a_tag_sen(sen_pred, sen_true, 9)==1:
#     obj_true+=1
#   else:
#     obj_false+=1
  
#   if arr_tag_in_a_tag_sen(sen_pred, sen_true, 10)==1:
#     obj_feature_true+=1
#   else:
#     obj_feature_false+=1

#   if arr_tag_in_a_tag_sen(sen_pred, sen_true, 11)==1:
#     pre_true+=1
#   else:
#     pre_false+=1

#   acc = homenumber_true+ street_true+ ward_true+ district_true+ province_true+ country_true+ postcode_true+ ner_true+ obj_true+ obj_feature_true+ pre_true
#   false = homenumber_false+ street_false+ ward_false+ district_false+ province_false+ country_false+ postcode_false+ ner_false+ obj_false+ obj_feature_false+ pre_false
#   print(acc)
#   print(false)
#   ti_le_dung = format(acc/(acc+false)*100, '.2f')
#   return ti_le_dung

In [ ]:
num_word = 42
num_char = 32
num_char_of_a_sen = num_word * num_char
char_embedding_dim = 137
batch_size = 64
output_lenght = 13
#===========================================================================================
char_input  = Input(shape=(num_word, num_char, char_embedding_dim), name="char_input")
char        = TimeDistributed(  Conv1D(filters = 8, kernel_size = 3, padding='same') ) (char_input)
char        = TimeDistributed(  Conv1D(filters = 8, kernel_size = 3, padding='same') ) (char)
char        = TimeDistributed(  MaxPool1D(pool_size = 2)  )  (char)
char        = TimeDistributed(  Conv1D(filters = 16, kernel_size = 3, padding='same') ) (char)
char        = TimeDistributed(  Conv1D(filters = 16, kernel_size = 3, padding='same') ) (char)
char        = TimeDistributed(  MaxPool1D(pool_size = 2)  )  (char)
char_out    = TimeDistributed(  Flatten()   )   (char)
#============================================================================================
word_input  = Input(shape=(num_word, 300), name="word_input")
word_out    = word_input
#============================================================================================
concatenated = concatenate([char_out, word_out])
# main BiLSTM model==========================================================================
out     = Bidirectional(LSTM(64, return_sequences=True, dropout=0.2, recurrent_dropout=0.2))(concatenated)
out     = TimeDistributed(Dense(output_lenght))(out)
output  = Activation('softmax')(out)
# crf = CRF(14, name="output")
# output = crf(out)
#===========================================================================================
m = Model(inputs=[char_input, word_input], outputs=output)
plot_model(m, "/content/drive/MyDrive/model/img/model_.png", show_shapes=True)
m.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
# m.compile(optimizer='adam', loss=crf.loss_function, metrics=[crf.accuracy, f1_m])
m.summary()






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
char_input (InputLayer)         (None, 42, 32, 137)  0                                            
__________________________________________________________________________________________________
time_distributed_1 (TimeDistrib (None, 42, 32, 8)    3296        char_input[0][0]                 
__________________________________________________________________________________________________
time_distributed_2 (TimeDistrib (None, 42, 32, 8)    200         time_distributed_1[0][0]         
__________________________________________________________________________________________________
time_distributed_3 (TimeDistrib (None, 42, 16, 8)    0           time

In [ ]:
# load model vừa train được với dữ liệu trc vào để train tiếp
# from keras.models import load_model
# m = load_model("/content/drive/MyDrive/model/model-use-rdr-4500-1-1-1.h5")
# m = load_model("/content/drive/MyDrive/CPoint model/mdl-last.h5")

In [ ]:
import keras

# cp_callback = keras.callbacks.ModelCheckpoint(filepath='/content/drive/MyDrive/CPoint model/mdl-last.h5', verbose=1, save_best_only=True)
# es_callback = keras.callbacks.EarlyStopping(monitor='acc', patience=5)

# character_train, character_test, word_train, word_test, tag_train, tag_test = train_test_split(char_embedd,word,tag, test_size=0.125)

# Train the model with the new callback
history = m.fit([char_embedd, word], tag,  
          epochs=10,
          batch_size = 64,
          validation_data=([val_char_embedd, val_word], val_tag),  
          # callbacks=[cp_callback,es_callback]
          )

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 3147 samples, validate on 450 samples
Epoch 1/10





3147/3147 [==============================] - 32s 10ms/step - loss: 1.1898 - acc: 0.7050 - val_loss: 0.7596 - val_acc: 0.7670
Epoch 2/10
3147/3147 [==============================] - 29s 9ms/step - loss: 0.6135 - acc: 0.8025 - val_loss: 0.4278 - val_acc: 0.8668
Epoch 3/10
3147/3147 [==============================] - 29s 9ms/step - loss: 0.3446 - acc: 0.8889 - val_loss: 0.2492 - val_acc: 0.9178
Epoch 4/10
3147/3147 [==============================] - 29s 9ms/step - loss: 0.2167 - acc: 0.9305 - val_loss: 0.1644 - val_acc: 0.9499
Epoch 5/10
3147/3147 [==============================] - 29s 9ms/step - loss: 0.1442 - acc: 0.9552 - val_loss: 0.1067 - val_acc: 0.9665
Epoch 6/10
3147/3147 [==============================] - 29s 9ms/step - loss: 0.0951 - acc: 0.9723 - val_loss: 0.0688 - val_acc: 0.9812
Epoch 7/10
3147/3147 [==================

In [ ]:
m.save("/content/drive/MyDrive/model/model-use-rdr-4500-1-1-1.h5")

In [ ]:
# import matplotlib.pyplot as plt
# # list all data in history
# print(history.history.keys())
# # summarize history for accuracy
# plt.plot(history.history['acc'])
# plt.plot(history.history['val_acc'])
# plt.title('model accuracy')
# plt.ylabel('accuracy')
# plt.xlabel('epoch')
# plt.legend(['train', 'test'], loc='upper left')
# plt.show()
# # summarize history for loss
# plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
# plt.title('model loss')
# plt.ylabel('loss')
# plt.xlabel('epoch')
# plt.legend(['train', 'test'], loc='upper left')
# plt.show()

In [ ]:
# from keras.models import load_model
# dependencies = {
#     'recall_m': recall_m,
#     'precision_m':precision_m,
#     'f1_m':f1_m
# }
# custom_objects=dependencies
# m = load_model("/content/drive/MyDrive/CPoint model/mdl-last_only_address.h5",custom_objects=dependencies)
# m = load_model("/content/drive/MyDrive/CPoint model/mdl-last.h5")

In [ ]:
# y_pred = m.predict([character_test, word_test])
# y_true = tag_test

In [ ]:
# print(y_pred.shape)
# print(y_true.shape)

In [ ]:
# from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
# y_pred = y_pred.reshape(y_pred.shape[0]*y_pred.shape[1], y_pred.shape[2])
# print(y_pred.shape)
# y_pred = np.argmax(y_pred,axis=1)

In [ ]:
# y_true = y_true.reshape(y_true.shape[0]*y_true.shape[1], y_true.shape[2])
# print(y_true.shape)
# y_true = np.argmax(y_true,axis=1)
# print(y_true.shape)

In [ ]:
# from sklearn.metrics import accuracy_score
# acc=0
# false=0
# for i in range(len(y_true)):
#   if y_true[i]!=0:
#     if y_true[i]==y_pred[i]:
#       acc+=1
#     else:
#       false+=1
# print(acc)
# print(false)
# print('accuracy = {}'.format(acc/(acc+false)) )

In [ ]:
# dict = []
# for element in y_true:
#   if element not in dict:
#     dict.append(element)
# print(dict)

In [ ]:
# cnf_matrix = confusion_matrix(y_true, y_pred, labels=dict)
# for i in range(len(cnf_matrix)):
#   for j in range(len(cnf_matrix)):
#     cnf_matrix[i][j] = round(cnf_matrix[i][j], 3)
# print(cnf_matrix)

In [ ]:
# normalized_confusion_matrix = cnf_matrix/cnf_matrix.sum(axis = 1, keepdims = True)
# for i in range(len(normalized_confusion_matrix)):
#   for j in range(len(normalized_confusion_matrix)):
#     normalized_confusion_matrix[i][j] = round(normalized_confusion_matrix[i][j], 3)
# print(normalized_confusion_matrix)

In [ ]:
# arr_to_cal_predict = []
# for i in range(len(cnf_matrix)):
#   tong= 0
#   for j in range(len(cnf_matrix)):
#     tong+=cnf_matrix[j][i]
#   arr_to_cal_predict.append(tong)
# print(arr_to_cal_predict)
# # lấy độ predict
# predict = []
# for i in range(len(cnf_matrix)):
#   tong=cnf_matrix[i][i]/arr_to_cal_predict[i]
#   predict.append(tong)
# print(predict)